In [67]:
pwd

'/Users/vaibhavsrinivasanday/Documents/Career/PythonStuff/Spotify/Spotify_Python/Spotify_automated/SpotifyPlayList'

In [68]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [69]:
pip install statistics

Note: you may need to restart the kernel to use updated packages.


In [70]:
import spotipy
import statistics as st
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import base64
import datetime
from urllib.parse import urlencode
import requests

In [71]:

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

In [72]:
client_id = 'fbfd8e76c5e849d8b438f9abcb9525a7'
client_secret = '19d9be5c3f5240368368cc38932de065'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [73]:
spotify = SpotifyAPI(client_id,client_secret)
a='Charlie Puth'
b='Adele'
c='John Mayer'
artist_ids=[]

artist_search=['Charlie Puth','Adele','John Mayer']
for i in range(0,len(artist_search)):
    d2=spotify.search({"artist":artist_search[i]},search_type="track")
    artist_ids.append(d2['tracks']['items'][0]['artists'][0]['uri'][15:])

print(artist_ids)

q=artist%3ACharlie+Puth&type=track
q=artist%3AAdele&type=track
q=artist%3AJohn+Mayer&type=track
['6VuMaDnrHyPL1p4EHjYLi7', '4dpARuHxo51G3z768sgnrY', '0hEurMDQu99nJRq8pTxO14']


In [74]:
#the id here is the artist's Spotify id
albums_info=[]
artists=artist_ids
for i in range(0,len(artists)):
    albums_info.append(sp.artist_albums(artists[i]))

album_names=[]
album_ids=[]
for k in range(0,len(albums_info)):
    for l in range(0,len(albums_info[k]['items'])):
        album_names.append(albums_info[k]['items'][l]['name'])
        album_ids.append(albums_info[k]['items'][l]['id'])

albums_final=np.array([album_names,album_ids]).T.tolist()

In [75]:
len(albums_final)
print(albums_final)

[['Voicenotes', '0mZIUXje90JtHxPNzWsJNR'], ['Nine Track Mind (Deluxe Edition)', '7E4Zm8sWQgkGyOm2v4d9D4'], ['Nine Track Mind (Deluxe Edition)', '1OSzM1OWqtTnmIJJQpn62Q'], ['Nine Track Mind', '5Nwsra93UQYJ6xxcjcE10x'], ['Nine Track Mind', '3kndSWeE2IYOrZEToZrHEV'], ['Nine Track Mind (Special Edition)', '5J4SS8wTmXdyIEVYjmHzpZ'], ['Nine Track Mind', '6E0p38TjcK490itZ1Ns7bD'], ['After All', '3kxT5SN1GQ3j3kS9NHOsYI'], ['Is It Just Me? (feat. Charlie Puth)', '6602j3WLi4WLHAjwGWW6T3'], ['Upside Down (feat. Charlie Puth)', '4rMBOhwN1GY0c2rpOoMU5L'], ['Upside Down (feat. Charlie Puth)', '4bRz24rWmRSSmcbZLsKXrI'], ['Hard On Yourself', '174SAjvfg1fkglW2S9vYkw'], ['Girlfriend (Haywyre Remix)', '4FUohmuUem6INx6ro34nCB'], ['Girlfriend (Live From Corden)', '4bsKJP1sB9EifTeAGF05rf'], ['Free (From Disney\'s "The One And Only Ivan")', '5hDTfO5ce0O7PiSbyLlsmT'], ['Girlfriend', '7AfbvLhUxPt0qOcXB2GTzY'], ['Summer Feelings (feat. Charlie Puth) [Acoustic]', '1HtmKNFx5Lpv2wxNH9to5d'], ['Summer Feelings (fea

In [76]:
#album_songs=[]
album_songs_name=[]
album_songs_id=[]
album_ids=[]
album_ids_new=[]
#Loop through to get the album ids for each album
for k in range(0,len(albums_final)):
    album_ids.append(albums_final[k][1])
    
for j in range(0,len(album_ids)):
    test=sp.album_tracks(album_ids[j])
    for i in range(0,len(test['items'])):
        album_songs_name.append(test['items'][i]['name'])
        album_songs_id.append(test['items'][i]['id'])
        
albums_songs_f=np.array([album_songs_name,album_songs_id]).T.tolist()
#print(len(album_songs_final))
#print(album_songs_final)
#print(np.array([album_songs_name,album_songs_id]).T.tolist())

In [77]:
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [name, album, artist, release_date, length, popularity, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

In [78]:
songs_details=[]
for i in range(0,len(albums_songs_f)):
    songs_details.append(getTrackFeatures(albums_songs_f[i][1]))
    

df = pd.DataFrame(songs_details, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
#df.to_csv("spotify_Charlie.csv", sep = ',')

In [79]:
df

,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,The Way I Am,Voicenotes,Charlie Puth,2018-05-11,186080,64,0.3140,0.755,0.7690,0.000002,0.0628,-5.658,0.1860,114.966,4
1,Attention,Voicenotes,Charlie Puth,2018-05-11,208786,81,0.1090,0.775,0.6130,0.000023,0.1340,-4.586,0.0542,100.066,4
2,LA Girls,Voicenotes,Charlie Puth,2018-05-11,197106,58,0.3590,0.869,0.6420,0.000001,0.0694,-5.030,0.0367,99.992,4
3,How Long,Voicenotes,Charlie Puth,2018-05-11,200853,73,0.2110,0.845,0.5610,0.000003,0.0383,-5.253,0.0778,109.974,4
4,Done for Me (feat. Kehlani),Voicenotes,Charlie Puth,2018-05-11,180493,68,0.1930,0.856,0.6320,0.000000,0.0688,-3.692,0.0740,112.009,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Comfortable - EP Version,Inside Wants Out,John Mayer,1999-09-24,299333,52,0.8060,0.328,0.1290,0.000004,0.1050,-13.523,0.0286,84.730,4
459,Quiet - EP Version,Inside Wants Out,John Mayer,1999-09-24,200533,39,0.7750,0.557,0.0695,0.000089,0.1040,-14.110,0.0383,103.230,3
460,Last Train Home - Ballad Version,Last Train Home (Ballad Version),John Mayer,2021-10-29,357705,62,0.4490,0.298,0.4360,0.371000,0.1000,-9.727,0.0512,78.207,4
461,Inside Friend (feat. John Mayer),Inside Friend (feat. John Mayer),Leon Bridges,2020-04-10,175432,63,0.4480,0.693,0.2590,0.000135,0.0995,-7.918,0.0372,88.972,4


In [80]:
###Connection to Google Sheets###

In [81]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint

In [82]:
scope = ["https://spreadsheets.google.com/feeds"
         ,'https://www.googleapis.com/auth/spreadsheets'
         ,"https://www.googleapis.com/auth/drive.file"
         ,"https://www.googleapis.com/auth/drive"]

In [83]:
creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)

In [126]:
client = gspread.authorize(creds)

In [127]:
sheet = client.open("SpotifyPythonData-PythonAPI").sheet1

In [128]:
sheet.batch_clear(["A2:V"])

{'spreadsheetId': '1QBEdAv6rnyyjGt4kVbdW8Ue8bcCSIyVxMtrv7nj-5vk',
 'clearedRanges': ['spotify_artists!A2:V1657']}

In [129]:
data = sheet.get_all_records()

In [130]:
insertRow = songs_details
sheet.insert_rows(insertRow,2)
#sheet.add_rows(insertRow)

{'spreadsheetId': '1QBEdAv6rnyyjGt4kVbdW8Ue8bcCSIyVxMtrv7nj-5vk',
 'updates': {'spreadsheetId': '1QBEdAv6rnyyjGt4kVbdW8Ue8bcCSIyVxMtrv7nj-5vk',
  'updatedRange': 'spotify_artists!A2:O464',
  'updatedRows': 463,
  'updatedColumns': 15,
  'updatedCells': 6945}}

list